# Evaluate SR output - hallucations and faithfullness

In [1]:
import sys

sys.path = ['/Users/jonathan/Projects/semantic-retrieval/python/src'] + sys.path

## Reproduce reports

In [2]:
# in semantic-retrieval/
# Reproduce net income report
!python python/src/semantic_retrieval/examples/financial_report/generate_report.py --retrieval-query="annual net income 2022" --data-extraction-prompt="annual net income in 2022" --overfetch-factor=20

python: can't open file '/Users/jonathan/Projects/semantic-retrieval/python/src/semantic_retrieval/examples/financial_report/python/src/semantic_retrieval/examples/financial_report/generate_report.py': [Errno 2] No such file or directory


In [3]:
# Reproduce covid report
# in semantic-retrieval/
!python python/src/semantic_retrieval/examples/financial_report/generate_report.py --retrieval-query="covid 19 impact" --data-extraction-prompt="covid 19 impact" --overfetch-factor=20 --log-level=DEBUG --client-name=client_a

python: can't open file '/Users/jonathan/Projects/semantic-retrieval/python/src/semantic_retrieval/examples/financial_report/python/src/semantic_retrieval/examples/financial_report/generate_report.py': [Errno 2] No such file or directory


## Example eval input: 
* Client portfolio database (local CSV)
* Intermediate SR results: raw retrieved document chunks
* SR final output saved in local files

In [14]:
import sys
import pandas as pd

from semantic_retrieval.retrieval.csv_retriever import CSVRetriever
# from semantic_retrieval.common import types
# from semantic_retrieval.evaluation.lib import (
#     LocalFileSystemGenLLMEvalDataset,
#     evaluate_llm_eval_dataset_local_filesystem,
#     evaluate_sample_local_filesystem,
#     local_filesystem_dataset_to_df,
#     # LocalFileSystemNumericalEvalDatasetConfig,
#     LocalFileSystemIDSetPairEvalDatasetConfig,
#     file_contents,
#     # IDSetPairEvalDataPathMuncher,
#     IDSetPairEvalDataset,
# )

import json

from semantic_retrieval.evaluation.metrics import jaccard_similarity

import glob
import os


ROOT_DATA_DIR = "../../../../../examples/example_data/financial_report/"

print("Ready")

Ready


In [15]:
from semantic_retrieval.examples.financial_report.eval_test_config import TEST_CASES

for tc in TEST_CASES:
    print("\nTest case:\n", tc)


Test case:
 ('net_income_vs_retrieved', <function test_1_2_data_muncher at 0x177d6c860>, 'artifacts/raw_retrieved_chunks_10k_net_income.json', 'artifacts/portfolio_10k_net_income_report.txt')

Test case:
 ('covid_vs_retrieved', <function test_1_2_data_muncher at 0x177d6c860>, 'artifacts/raw_retrieved_chunks_10k_covid.json', 'artifacts/portfolio_10k_covid_report.txt')

Test case:
 ('net_income_e2e', <function test_3_4_data_muncher at 0x177d6c9a0>, 'portfolios/sarmad_portfolio.csv', 'artifacts/portfolio_10k_net_income_report.txt')

Test case:
 ('covid_e2e', <function test_3_4_data_muncher at 0x177d6c9a0>, 'portfolios/sarmad_portfolio.csv', 'artifacts/portfolio_10k_covid_report.txt')


In [16]:
from semantic_retrieval.functional.functional import result_reduce_list_separate

from semantic_retrieval.examples.financial_report import lib as fr_lib

evaluation_params_list = [
    await fr_lib.test_case_to_sample_eval_params(tc, ROOT_DATA_DIR) for tc in TEST_CASES
]

print("Structured eval params:\n")

# Separate out the valid test params from the error ones.
evaluation_params_valid, evaluation_params_errs = result_reduce_list_separate(
    evaluation_params_list
)

for eval_params in evaluation_params_valid:
    print(eval_params)


print(f"\n\n\nEvaluation params with errors (empty list is good): {evaluation_params_errs}")

Structured eval params:


SampleEvaluationParams:
	self.output_sample={'JNJ', 'NVDA', 'UNH', 'AMZN', 'AAPL', 'PEP', 'MRK'}
	self.evaluation_fn=functools.partial(<function idset_jaccard_with_gt at 0x1770393a0>, gt={'JNJ', 'NVDA', 'UNH', 'AMZN', 'AAPL', 'PEP', 'MRK'}, name='net_income_vs_retrieved')

SampleEvaluationParams:
	self.output_sample={'JNJ', 'NVDA', 'UNH', 'AMZN', 'AAPL', 'PEP', 'MRK'}
	self.evaluation_fn=functools.partial(<function idset_jaccard_with_gt at 0x1770393a0>, gt={'JNJ', 'NVDA', 'UNH', 'AMZN', 'AAPL', 'PEP', 'MRK'}, name='covid_vs_retrieved')

SampleEvaluationParams:
	self.output_sample={'JNJ', 'NVDA', 'UNH', 'AMZN', 'AAPL', 'PEP', 'MRK'}
	self.evaluation_fn=functools.partial(<function idset_jaccard_with_gt at 0x1770393a0>, gt={'JNJ', 'NVDA', 'UNH', 'AMZN', 'AAPL', 'PEP', 'MRK'}, name='net_income_e2e')

SampleEvaluationParams:
	self.output_sample={'JNJ', 'NVDA', 'UNH', 'AMZN', 'AAPL', 'PEP', 'MRK'}
	self.evaluation_fn=functools.partial(<function idset_jaccard_with_gt

In [17]:
from semantic_retrieval.evaluation.lib import (
    eval_res_to_df,
    evaluate,
)

# Helper function to pretty print all the results
def _print(df: pd.DataFrame) -> None:
    return display(df.set_index(["name", "interpretation"]))




# Run `evaluate()` on the valid params.
eval_res = evaluate(evaluation_params_valid)

# Print results
df_eval_res = eval_res.map(eval_res_to_df)
df_eval_res.map(_print)

,,value
name,interpretation,
net_income_vs_retrieved,Jaccard similarity with ground truth,1.0
covid_vs_retrieved,Jaccard similarity with ground truth,1.0
net_income_e2e,Jaccard similarity with ground truth,1.0
covid_e2e,Jaccard similarity with ground truth,1.0


Ok(None)